In this Python worflow we explore the Montreal Bixi biking data set for the year 2017 https://www.kaggle.com/aubertsigouin/biximtl/data

We have additionally enriched this data set with the biking distance/duration available via Google map API as gmdata2017

Our objective is to predict the "trip duration", given the distance between two stations.

We are using the Pandas package with an explicitly optimized setting for database connection to transfer data.

Import required pacakges.

In [1]:
import pandas.io.sql as psql;
import pandas as pd;
import pymonetdb.sql;

Database connection information. We will try to explicitly optimize the database connection by providing a very high value for the data buffer.

In [2]:
host='cerberus'; dbname='bixi'; user='bixi'; passwd='bixi'; databuffersize=1000000;

Connect to the database.

In [3]:
con = pymonetdb.Connection(dbname,hostname=host,username=user,password=passwd,autocommit=True);
con.replysize = databuffersize;

Let us find out what are the tables available in the database.

Python DB API spec does not provide a mechanism to list the tables in the database, so it is left to the users to write a query depending on the RDBMS vendor.

In [4]:
tblListSQL = \
        "SELECT t.name as tableName " \
        "FROM sys.tables t " \
        "  INNER JOIN sys.schemas s " \
        "    ON t.schema_id = s.id " \
        "WHERE s.name = '{}'"\
        ";"

tables = psql.read_sql_query(sql=tblListSQL.format('bixi'), con=con);
print(tables);

      tablename
0  stations2017
1  tripdata2017
2    gmdata2017
3          temp


Let us take a peek into tripdata2017.
For this purpose, we will create a pandas dataframe for tripdata2017 that we can reuse later.

In [5]:
tripdata2017 = pd.DataFrame(psql.read_sql_query('SELECT * FROM tripdata2017;', con));
print(tripdata2017.head());
print(tripdata2017.describe());

   id             starttm  stscode               endtm  endscode  duration  \
0   0 2017-04-15 00:00:00     7060 2017-04-15 00:31:00      7060      1841   
1   1 2017-04-15 00:01:00     6173 2017-04-15 00:10:00      6173       553   
2   2 2017-04-15 00:01:00     6203 2017-04-15 00:04:00      6204       195   
3   3 2017-04-15 00:01:00     6104 2017-04-15 00:06:00      6114       285   
4   4 2017-04-15 00:01:00     6174 2017-04-15 00:11:00      6174       569   

   ismember  
0         1  
1         1  
2         1  
3         1  
4         1  
                 id       stscode      endscode      duration      ismember
count  4.018721e+06  4.018721e+06  4.018721e+06  4.018721e+06  4.018721e+06
mean   2.009360e+06  6.324815e+03  6.319868e+03  8.374505e+02  7.992535e-01
std    1.160105e+06  3.758616e+02  3.832840e+02  6.577148e+02  4.005588e-01
min    0.000000e+00  5.002000e+03  5.002000e+03  6.100000e+01  0.000000e+00
25%    1.004680e+06  6.105000e+03  6.092000e+03  3.820000e+02  1.00

So we have 4 million + records in tripdata2017. Also, the station codes are labels. We may have to enrich this information. Let us take a look at the contents of stations2017.

In [6]:
stations2017 = pd.DataFrame(psql.read_sql_query('SELECT * FROM stations2017;', con));
print(stations2017.head());
print(stations2017.describe());

   scode                      sname  slatitude  slongitude  sispublic
0   7060  "de l'Église / de Verdun"  45.463001  -73.571569          1
1   6173         "Berri / Cherrier"  45.519088  -73.569509          1
2   6203   "Hutchison / Sherbrooke"  45.507810  -73.572080          1
3   6204        "Milton / Durocher"  45.508144  -73.574772          1
4   6104    "Wolfe / René-Lévesque"  45.516818  -73.554188          1
              scode   slatitude  slongitude   sispublic
count    546.000000  546.000000  546.000000  546.000000
mean    6412.743590   45.519109  -73.582622    0.983516
std      405.396125    0.027919    0.027344    0.127442
min     5002.000000   45.430740  -73.670634    0.000000
25%     6143.500000   45.501726  -73.599579    1.000000
50%     6305.500000   45.523103  -73.577413    1.000000
75%     6722.750000   45.538784  -73.564608    1.000000
max    10002.000000   45.582757  -73.495067    1.000000


This is good, we have the longitude and latitude associated with each station, which can be used to enrich the tripdata.

Since we have 546 stations, this gives the possibility of 546 x 546 = 298116 possible scenarios for trips. However, we need not be concerned with trips that started and ended at the same station as those are noise. Also, to weed out any further fluctuations in the input data set, we will limit ourselves to only those station combinations which has at the least 50 trips.

For this purpose, we will use some relational-API like feature of Pandas.

In [7]:
freqStations = tripdata2017.where(tripdata2017['stscode'] != tripdata2017['endscode']).dropna();
freqStations = pd.DataFrame({'numtrips' : freqStations.groupby(['stscode', 'endscode']).size()}).reset_index();
freqStations = freqStations.where(freqStations['numtrips'] >= 50).dropna();
print(freqStations.head());
print(freqStations.describe());

     stscode  endscode  numtrips
4     5002.0    5007.0      99.0
44    5003.0    5007.0      86.0
90    5004.0    5007.0     200.0
123   5005.0    5007.0     180.0
172   5006.0    5007.0     221.0
            stscode      endscode      numtrips
count  19300.000000  19300.000000  19300.000000
mean    6302.127358   6291.781295    116.905855
std      349.880754    351.217448    117.250651
min     5002.000000   5002.000000     50.000000
25%     6100.000000   6078.000000     62.000000
50%     6194.000000   6180.000000     81.000000
75%     6350.000000   6362.000000    125.000000
max    10002.000000  10002.000000   2200.000000


We can see that there are 19,300 station combinations that is of interest to us. Next we will include the longitude and latitude information of the start and end stations.

In [8]:
freqStationsCord = pd.merge(freqStations, stations2017, left_on='stscode', right_on='scode') \
                    .loc[:,['stscode', 'endscode', 'numtrips', 'slatitude', 'slongitude']] \
                    .rename(index=str, columns={'slatitude':'stlat', 'slongitude':'stlong'});
freqStationsCord = pd.merge(freqStationsCord, stations2017, left_on='endscode', right_on='scode') \
                    .loc[:,['stscode', 'endscode', 'numtrips', 'stlat', 'stlong', 'slatitude', 'slongitude']] \
                    .rename(index=str, columns={'slatitude':'enlat', 'slongitude':'enlong'});

print(freqStationsCord.head());

  stscode endscode  numtrips      stlat     stlong      enlat     enlong
0    5002     5007      99.0  45.533703 -73.515283  45.523854 -73.519677
1    5003     5007      86.0  45.529512 -73.517691  45.523854 -73.519677
2    5004     5007     200.0  45.539824 -73.508752  45.523854 -73.519677
3    5005     5007     180.0  45.536378 -73.512642  45.523854 -73.519677
4    5006     5007     221.0  45.537226 -73.495067  45.523854 -73.519677


It would be easier if we can translate the coordinates to a distance metric. Python's geopy module supports this computation using Vincenty's formula. This provides us with a distance as crow flies between two coordiantes. This might be a reasonable approximation of actual distance travelled in a trip.

In [9]:
import geopy.distance;     #We will use this module to compute distance.
def computeDist(trip):
    #These are the inputs to Vincenty's formula.
    stlat = trip['stlat']; stlong = trip['stlong']; enlat = trip['enlat']; enlong = trip['enlong'];
    #populate the distance metric using longitude/latitude of coordinates.
    return int(geopy.distance.distance((stlat,stlong), (enlat,enlong)).meters);

freqStationsDist = pd.DataFrame(freqStationsCord);
freqStationsDist['vdistm'] = freqStationsDist.apply(computeDist, axis=1);
print(freqStationsDist.head());

  stscode endscode  numtrips      stlat     stlong      enlat     enlong  \
0    5002     5007      99.0  45.533703 -73.515283  45.523854 -73.519677   
1    5003     5007      86.0  45.529512 -73.517691  45.523854 -73.519677   
2    5004     5007     200.0  45.539824 -73.508752  45.523854 -73.519677   
3    5005     5007     180.0  45.536378 -73.512642  45.523854 -73.519677   
4    5006     5007     221.0  45.537226 -73.495067  45.523854 -73.519677   

   vdistm  
0    1147  
1     647  
2    1969  
3    1496  
4    2429  


We can next enrich our trip data set with the distance information by joining these computed distances with each trip.

In [10]:
tripData = pd.merge(tripdata2017, freqStationsDist, on=['stscode', 'endscode']) \
             .loc[:,['id', 'duration', 'vdistm']];
print(tripData.head());
print(tripData.describe());

      id  duration  vdistm
0      2       195     213
1   8213      3152     213
2  22734       124     213
3  34304        97     213
4  54111        92     213
                 id      duration        vdistm
count  2.256283e+06  2.256283e+06  2.256283e+06
mean   2.003455e+06  6.304499e+02  1.369893e+03
std    1.162663e+06  5.334432e+02  9.212933e+02
min    2.000000e+00  6.100000e+01  4.700000e+01
25%    9.945115e+05  2.980000e+02  7.110000e+02
50%    2.005024e+06  4.820000e+02  1.117000e+03
75%    3.010572e+06  7.930000e+02  1.755000e+03
max    4.018721e+06  7.199000e+03  9.074000e+03


So we have trip duration for each trip and the distance as crow flies, between the two stations involved in the trip.

Also, we have about 2 million trips for which we have distance between stations metric. Given that there are only a few thousand unique values for distance, we might want to keep some values of distance apart for testing. For this purpose, we will first get distinct values for distance and then sort it.

In [11]:
uniqueTripDist = tripData.loc[:,['vdistm']].drop_duplicates().sort_values(by=['vdistm']);

print(uniqueTripDist.head());
print(uniqueTripDist.tail());
print(uniqueTripDist.describe());

         vdistm
2244879      47
56654        71
1898286      81
1776285      85
1011539      88
         vdistm
908190     8529
1841645    8752
1971339    8860
1761199    9031
2197167    9074
            vdistm
count  3652.000000
mean   2203.772453
std    1386.237638
min      47.000000
25%    1095.750000
50%    2013.500000
75%    3068.250000
max    9074.000000


We will keep some data apart for testing. A rule of thumb is 30%. The neat trick below sets apart 33%, across the entire range of distance values. close enough.

In [12]:
testTripDist = uniqueTripDist[::3];
print(testTripDist.head());
print(testTripDist.tail());
print(testTripDist.describe());

         vdistm
2244879      47
1776285      85
545957      110
1783        126
1462413     148
         vdistm
2238694    6796
1074925    7057
2192277    7530
1841645    8752
2197167    9074
            vdistm
count  1218.000000
mean   2205.059113
std    1391.487164
min      47.000000
25%    1095.750000
50%    2013.500000
75%    3068.250000
max    9074.000000


Now let us get the remaining values for distances to be used for training.

In [13]:
trainTripDist = uniqueTripDist[~uniqueTripDist['vdistm'].isin(testTripDist['vdistm'])];
print(trainTripDist.head());
print(trainTripDist.tail());
print(trainTripDist.describe());

         vdistm
56654        71
1898286      81
1011539      88
775871       94
1299438     120
         vdistm
812754     7307
2074486    7650
908190     8529
1971339    8860
1761199    9031
            vdistm
count  2434.000000
mean   2203.128595
std    1383.889269
min      71.000000
25%    1096.250000
50%    2013.500000
75%    3067.750000
max    9031.000000


Let us now extract the fields of interest to us for the training data, which is just the distance of each trip and it's duration

In [14]:
trainData = tripData[tripData['vdistm'].isin(trainTripDist['vdistm'])];
trainData = trainData.loc[:, ['vdistm', 'duration']];
print(trainData.head());
print(trainData.tail());
print(trainData.describe());

   vdistm  duration
0     213       195
1     213      3152
2     213       124
3     213        97
4     213        92
         vdistm  duration
2256278    1433       472
2256279    1433       511
2256280    1433       774
2256281    1433       517
2256282    1433       579
             vdistm      duration
count  1.503408e+06  1.503408e+06
mean   1.370759e+03  6.294150e+02
std    9.262827e+02  5.360811e+02
min    7.100000e+01  6.100000e+01
25%    7.060000e+02  2.950000e+02
50%    1.109000e+03  4.800000e+02
75%    1.774000e+03  7.930000e+02
max    9.031000e+03  7.199000e+03


As the values are huge, we should normalize the data attributes. First get the max values for these attributes.

In [15]:
maxdist = uniqueTripDist['vdistm'].max();
print(maxdist);
maxduration = tripData['duration'].max();
print(maxduration);

9074
7199


Now let us normalize the training data. As we are working with integer data, we will also have to convert it to float. That can be accomplished by multiplying with 1.0.

In [16]:
trainData['vdistm'] = trainData['vdistm']/maxdist;
trainData['duration'] = trainData['duration']/maxduration;

print(trainData.head());
print(trainData.tail());

     vdistm  duration
0  0.023474  0.027087
1  0.023474  0.437839
2  0.023474  0.017225
3  0.023474  0.013474
4  0.023474  0.012780
           vdistm  duration
2256278  0.157924  0.065565
2256279  0.157924  0.070982
2256280  0.157924  0.107515
2256281  0.157924  0.071816
2256282  0.157924  0.080428


Our linear regression equation is of the form.

dur = a + b*dist

we will re-organize the training data set to fit this format and also setup our initial parameters for a and b.

In [17]:
trainDataSet = trainData.loc[:, ['vdistm']];
trainDataSet.insert(0, 'x0', 1);
print(trainDataSet.head());
trainDataSetDuration = trainData.loc[:, ['duration']];
print(trainDataSetDuration.head());
#With pandas data frames, to do matrix multiplication, the column names should match.
params = pd.DataFrame({'x0':[1], 'vdistm':[1]}, index=['duration']);
print(params);

   x0    vdistm
0   1  0.023474
1   1  0.023474
2   1  0.023474
3   1  0.023474
4   1  0.023474
   duration
0  0.027087
1  0.437839
2  0.017225
3  0.013474
4  0.012780
          vdistm  x0
duration       1   1


Let us try to run a prediction using these parameters.

In [18]:
pred = trainDataSet.dot(params.T);
print(pred.head());

   duration
0  1.023474
1  1.023474
2  1.023474
3  1.023474
4  1.023474


We need to compute the squared error for the predictions. Since we will be reusing them, we might as well store it as a function.

In [19]:
def squaredErr(actual, predicted):
    return ((predicted-actual)**2).sum()/(2*(actual.shape[0]));

Let us see what is the error for the first iteration.

In [20]:

sqerr = squaredErr(trainDataSetDuration, pred);
print(sqerr);

duration    0.569487
dtype: float64


We need to perform a gradient descent based on the squared errors. We will write another function to perform this.

In [21]:
def gradDesc(actual, predicted, indata):
    return (predicted-actual).T.dot(indata) / actual.shape[0];

Let us update our params using gradient descent using the error we got. We also need to use a learning rate, alpha (arbitrarily chosen).

In [22]:
alpha = 0.1;

params = params - alpha * gradDesc(trainDataSetDuration, pred, trainDataSet);
print(params);

            vdistm        x0
duration  0.983306  0.893637


Now let us try to use the updated params to train the model again and see if the error is decreasing.

In [23]:
pred = trainDataSet.dot(params.T);
print(pred.head());
sqerr = squaredErr(trainDataSetDuration, pred);
print(sqerr);

   duration
0  0.916718
1  0.916718
2  0.916718
3  0.916718
4  0.916718
duration    0.459497
dtype: float64


Before we proceed, may be we should check if google maps API's distance metric gives a better learning rate. Let us see what fields we can use from Google

In [24]:
gmdata2017 = pd.DataFrame(psql.read_sql_query('SELECT * FROM gmdata2017;', con));
print(gmdata2017.head());
print(gmdata2017.describe());

   stscode  endscode  gdistm  gduration
0     6406      6052    3568        596
1     6050      6406    3821        704
2     6148      6173    1078        293
3     6110      6114    1319        337
4     6123      6114     725        177
            stscode      endscode        gdistm     gduration
count  19516.000000  19516.000000  19516.000000  19516.000000
mean    6302.622361   6291.736729   2079.293093    516.190869
std      350.305819    350.469248   1345.629279    299.191977
min     5002.000000   5002.000000     18.000000      4.000000
25%     6100.000000   6078.000000   1118.000000    300.000000
50%     6194.000000   6180.000000   1766.000000    459.000000
75%     6350.000000   6362.000000   2711.000000    671.000000
max    10002.000000  10002.000000  14530.000000   3083.000000


We can build a new data set for the trips between frequently used station combination that includes google's distance.

In [25]:
gtripData = pd.merge(gmdata2017, tripdata2017, on=['stscode', 'endscode']) \
              .loc[:,['stscode', 'endscode', 'id', 'duration', 'gdistm', 'gduration']];
gtripData = pd.merge(gtripData, freqStations, on=['stscode', 'endscode']) \
              .loc[:,['id', 'duration', 'gdistm', 'gduration']];
print(gtripData.head());
print(gtripData.describe());

     id  duration  gdistm  gduration
0  4152      1561    3568        596
1  4154      1482    3568        596
2  4985       965    3568        596
3  4986       968    3568        596
4  6354      1376    3568        596
                 id      duration        gdistm     gduration
count  2.256283e+06  2.256283e+06  2.256283e+06  2.256283e+06
mean   2.003455e+06  6.304499e+02  1.834732e+03  4.546805e+02
std    1.162663e+06  5.334432e+02  1.236259e+03  2.733622e+02
min    2.000000e+00  6.100000e+01  4.800000e+01  1.100000e+01
25%    9.945115e+05  2.980000e+02  9.600000e+02  2.570000e+02
50%    2.005024e+06  4.820000e+02  1.497000e+03  3.940000e+02
75%    3.010572e+06  7.930000e+02  2.359000e+03  5.910000e+02
max    4.018721e+06  7.199000e+03  1.453000e+04  3.083000e+03


Google also provides its estimated duration for the trip. We will have to see in the end if our trained model is able to predict the trip duration better than google's estimate. So we will also save Google's estimate for the trip duration for that comparison.

Next up, we need to format this dataset the same way we did the first one.

In [26]:
guniqueTripDist = gtripData.loc[:,['gdistm']].drop_duplicates().sort_values(by=['gdistm']);
gtestTripDist = guniqueTripDist[::3];

gtrainTripDist = guniqueTripDist[~guniqueTripDist['gdistm'].isin(gtestTripDist['gdistm'])];
gtrainData = gtripData[gtripData['gdistm'].isin(gtrainTripDist['gdistm'])];
gtrainData = gtrainData.loc[:, ['gdistm', 'duration']];

gmaxdist = guniqueTripDist['gdistm'].max();
print(gmaxdist);
gmaxduration = gtripData['duration'].max();
print(gmaxduration);
gtrainData['gdistm'] = gtrainData['gdistm']/gmaxdist;
gtrainData['duration'] = gtrainData['duration']/gmaxduration;

gtrainDataSet = gtrainData.loc[:, ['gdistm']];
gtrainDataSet.insert(0, 'x0', 1);
gtrainDataSetDuration = gtrainData.loc[:, ['duration']];
gparams = pd.DataFrame({'x0':[1], 'gdistm':[1]}, index=['duration']);

14530
7199


Let us see how the error rate is progressing for the new dataset.

In [27]:
gpred = gtrainDataSet.dot(gparams.T);
gsqerr = squaredErr(gtrainDataSetDuration, gpred);
print(gsqerr);
gparams = gparams - alpha * gradDesc(gtrainDataSetDuration, gpred, gtrainDataSet);
gpred = gtrainDataSet.dot(gparams.T);
gsqerr = squaredErr(gtrainDataSetDuration, gpred);
print(gsqerr);

duration    0.541968
dtype: float64
duration    0.437908
dtype: float64


It looks like using Google maps' distance is giving us a slight advantage. That makes sense, since Vincenty's formula computes distances as a crow flies, where as Google maps' distance metric is based on the actual road network distances. Better data gives better prediction results !

We are done with the feature selection and feature engineering phase for now.

Next we will proceed to train our linear regression model using the training data set.

Meanwhile, we will also let it printout the error rate at frequent intervals so that we know it is decreasing.

In [28]:
for i in range(0, 1000):
    gpred = gtrainDataSet.dot(gparams.T);
    gparams = gparams - alpha*gradDesc(gtrainDataSetDuration, gpred, gtrainDataSet);
    if((i+1)%100 == 0):
        print("Error rate after {} iterations is {}".format(i+1, squaredErr(gtrainDataSetDuration, gpred)))
    
print(gparams);
gsqerr = squaredErr(gtrainDataSetDuration, gpred);
print(gsqerr);

Error rate after 100 iterations is duration    0.002415
dtype: float64
Error rate after 200 iterations is duration    0.002352
dtype: float64
Error rate after 300 iterations is duration    0.002297
dtype: float64
Error rate after 400 iterations is duration    0.00225
dtype: float64
Error rate after 500 iterations is duration    0.002209
dtype: float64
Error rate after 600 iterations is duration    0.002173
dtype: float64
Error rate after 700 iterations is duration    0.002142
dtype: float64
Error rate after 800 iterations is duration    0.002115
dtype: float64
Error rate after 900 iterations is duration    0.002092
dtype: float64
Error rate after 1000 iterations is duration    0.002072
dtype: float64
            gdistm        x0
duration  0.671629  0.002959
duration    0.002072
dtype: float64


Let us see how our model performs in predictions against the test data set we had kept apart.

In [29]:
gtestData = gtripData[gtripData['gdistm'].isin(gtestTripDist['gdistm'])];
gtestData = gtestData.loc[:, ['gdistm', 'duration', 'gduration']];
gtestData['gdistm'] = gtestData['gdistm']/gmaxdist;
gtestData['duration'] = gtestData['duration']/gmaxduration;
gtestDataSet = gtestData.loc[:, ['gdistm']];
gtestDataSet.insert(0, 'x0', 1);
gtestDataSetDuration = gtestData.loc[:, ['duration']];

gtestpred = gtestDataSet.dot(gparams.T);

gtestsqerr1 = squaredErr(gtestDataSetDuration*gmaxduration, gtestpred*gmaxduration);
print(gtestsqerr1);

duration    99215.984246
dtype: float64


We would also like to check how the duration provided by Google maps' API hold up to the test data set.

In [30]:
gtestsqerr2 = squaredErr(gtestDataSetDuration*gmaxduration, gtestData.loc[:,['gduration']] \
                         .rename(columns={'gduration':'duration'}));
print(gtestsqerr2);

duration    111763.379836
dtype: float64


So yes, our model is able to do a better job.